# Getting Started

*ide49* offers a range of features including a code editor, compilers (e.g. to compile MicroPython), an MQTT broker, etc. 

All interaction with microcontrollers takes place in the [*Jupyter Lab*](https://jupyter.org/) service. *Jupyter* is a popular interactive Python programming environment. You can find many online tutorials and videos. Below we quickly go over the main features with special focus on the customizations for programming microcontrollers.

## Jupyter Lab

To access *Jupyter Lab* in *ide49*, go to https://iot49.local/jupyter (change the name if you modified the DNS name of the Raspberry Pi) or from the *Jupyter* link of *ide49* dashboard at https://iot49.local.

You will be presented with the page below. 

![Jupyter](figures/jupyter_annotated.png)

For illustration, the key features are highlighted. The main features include a *Python 3 Kernel* for executing code on the Raspberry Pi, a *Terminal* for running shell commands, and, the **IoT Kernel** for programming microcontrollers. Click on that button to get a new window:

To start an *IoT Kernel*, click on its icon to get:

![IoT Kernel](figures/iot_kernel_annotated.png)

Like all *Jupyter* kernels, you type code into *cells*, and click `shift-return` (return while holding down the shift key) to execute it. 

For example:

```{image} figures/no_device_connected.png
:width: 250px
```

The error is not surprising, since, well, we have not yet connected a microcontroller to the Raspberry Pi.

Like other *Jupyter* kernels, it implements several *magics*; lines that start with a percent sign `%` in the first column. To get a list of magics available in the *IoT Kernel*, execute `%lsmagic%` by typing it into an empty cell and hitting `shift-return`:

In [1]:
%lsmagic

Line Magic:    -h shows help (e.g. %discover -h)
  %cat         Print contents of named file on microcontroller
  %cd          Change the working directory.
  %connect     Connect to device
  %cp          Copy files between host and microcontroller.
  %discover    Discover available devices
  %gettime     Query microcontroller time
  %loglevel    Set logging level.
  %lsmagic     List all magic functions.
  %mkdirs      Create all directories specified by the path, as needed.
  %name        Name of currently connected microcontroller.
  %pip         Install packages from PyPi
  %platform    sys.platform of currently connected device.
  %rdiff       Show differences between microcontroller and host directories
  %register    Register device
  %rlist       List files on microcontroller
  %rm          Delete files relative to path.
  %rsync       Synchronize microcontroller to host directories
  %softreset   Reset microcontroller. Similar to pressing the reset button.
  %store       Copy 

To get additonal information about a magic, run it with the `-h` flag:

In [1]:
%%host -h

usage: %%host [-h]

Pass cell to host (cPython) for evaluation.

optional arguments:
  -h, --help  show this help message and exit


In [2]:
%%host

import sys
print(sys.platform)

linux


Code preceded by the `%%host` magic is submitted to the Raspberry Pi for execution. See [iot-kernel](iot-kernel) for more information.

## MicroPython

Connect a microcontroller to one of the USB ports of the Raspberry Pi & [install MicroPython](flash.ipynb). If MicroPython is installed already, you can skip this step.

In [1]:
%discover

50:02:91:a1:a7:2c  serial:///dev/ttyUSB0  


In [ ]:
The `%discover` magic searches all ports for available devices. For each device it reports a hexadecimal string that uniquely identifies the processors (hence you'll get a different number) and the port it is connected to.

Connection is initiated with the `%connect` magic.

In [ ]:
%connect 50:02:91:a1:a7:2c

Code typed into notebook cells is sent to the microcontroller for execution and results reported back. For example:

In [4]:
for i in range(6):
    print(i, i**5)

0 0
1 1
2 32
3 243
4 1024
5 3125


`%%connect` with two percent signs runs a code sequence on multiple processors:

In [4]:
%%connect --host 50:02:91:a1:a7:2c
                    
import sys
print("platform: {:7} implementation: {}".format(sys.platform, sys.implementation.name))


----- HOST

platform: linux   implementation: cpython

----- 50:02:91:a1:a7:2c

platform: esp32   implementation: micropython


We are indeed connected to MicroPython.

## Naming devices

The string `50:02:91:a1:a7:2c` is the *UID* of the microcontroller, a unique identifier programmed into the controller by the manufacturer. The `%discover` magic obtains it by running the following code snippet on the controller:

```python
uid = bytes(6)
try:
    import machine
    uid = machine.unique_id()
except:
    import microcontroller
    uid = microcontroller.cpu.uid
print(":".join("{:02x}".format(x) for x in uid), end="")
```

The *IoT Kernel* uses *UIDs* to unambiguously distinguish devices. Identifying microcontrollers with hexadecimal strings is useful for keeping track of devices (I have an entire drawer full), but about as informative and convenient to remember as a phone number. 

Fortunately it is easy to associate the device with a "name", by simply recording this information in a file and saving it to folder `~/iot49/devices`. The `%%writefile` magic does this for us:

In [5]:
%%writefile ~/iot49/devices/my_microcontroller.yaml
pico:
    uid: 50:02:91:a1:a7:2c

Writing /home/iot/iot49/devices/my_microcontroller.yaml


Change the `uid` to match your microcontroller and the word `pico` to the name you want to give. The filename is not important, as long as it ends with `.yaml`. 

After this, `%discover` reports the name instead of the *UID*:

In [5]:
%discover

pico  serial:///dev/ttyUSB0  


## Getting a REPL prompt

If you rather use the REPL to program your microcontroller, launch a Terminal (click the blue `+` icon at the upper left of jupyter lab and then the Terminal icon). In the terminal type

```bash
picocom /dev/ttyUSB0  -b115200
```

```{toggle}
![REPL](figures/repl.png)
```